# ⚽ **HOL: Eredivisie Prediction** 🥇
### Notebook - Data Transformation - 2/4

---

### What We'll Do:
1. **Data Ingestion**: Fetch Eredivisie data from the GitHub repository.
2. -> **Data Transformation**: Utilize Snowpark DataFrames for data preparation and analysis.
3. **Model Training**: Train model and store it in the Snowflake Model Registry
4. **Prediction**: Predict who is going to win Eredivisie 2024/2025


![image](https://i.gifer.com/embedded/download/BiCu.gif)



## Step 2: Data Transformation
---

In the next cells, we'll start working on Data Engineering Tasks, joining and normalizing the data to extract meaningful features and leveraging Snowpark Dataframe for the data manipulation.


In nutshell, we'll follow perform: 

- Data Cleaning
- Data Transformation
- Feature Engineering
- Preparing Feature Table

Let's get it done! 

### Setup

Before using this notebook, ensure that you have imported the following packages by click on the top right "Packages" button and restart the notebook:

- `snowflake-snowpark-python`


In [ ]:
# Import python packages
import streamlit as st
from snowflake.snowpark.functions import col, to_date, datediff, lag, when, lit, sum as sum_, concat, count, rank, countDistinct, year, when, row_number, datediff, max as max_
from snowflake.snowpark.window import Window
import snowflake.snowpark.functions as F
import snowflake.snowpark as snowpark

from snowflake.snowpark.window import Window
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()
from snowflake.snowpark.version import VERSION
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### Data Cleaning, Enrichment & Normalization

We'll be fixing the following issues in the dataset:
- Team Names mismatch
- Calculating season
- Trimming Values

In [ ]:
# Get History tables
EREDIVISIE_HISTORY_df = session.table('EREDIVISIE_PREDICTION.PUBLIC.EREDIVISIE_HISTORY')

In [ ]:
def check_team_name_uniqueness(df):
    # Step 1: Analyze teams

    home_teams = df.select("HOMETEAM").distinct().sort("HOMETEAM")
    away_teams = df.select("AWAYTEAM").distinct().sort("AWAYTEAM")
    
    #print("Home teams:")
    #home_teams.show(50)
    st.text(f"Number of unique home teams: {home_teams.count()}")
    #print("\nAway teams:")
    #away_teams.show(50)
    st.text(f"Number of unique away teams: {away_teams.count()}")
    
    all_teams = home_teams.union(away_teams).distinct()
    #print("\nAll unique teams:")
    st.text(f"Number of all unique teams: {all_teams.count()}")
    st.dataframe(all_teams.sort(col("HOMETEAM")))
    
check_team_name_uniqueness(EREDIVISIE_HISTORY_df)

In [ ]:
def add_season_columns(df):
    # Convert DATE column to date type
    df = df.withColumn("DATE", to_date(col("DATE"), "dd/MM/yyyy"))
    
    # Create a window specification sorted by date
    window_spec = Window.orderBy("DATE")
    
    # Calculate the difference in days from the previous row
    df = df.withColumn("days_diff", datediff("day", lag("DATE").over(window_spec), col("DATE")))
    
    # Determine the season start (1 for first row, 1 when diff > 50 days)
    df = df.withColumn("season_start", 
                       when(col("days_diff").isNull() | (col("days_diff") > 50), 1)
                       .otherwise(0))
    
    # Calculate the cumulative sum of season_start to get the season number
    df = df.withColumn("season_number", sum_(col("season_start")).over(window_spec))
    
    # Create the season column (e.g., "1993-1994")
    df = df.withColumn("season", 
                       concat(
                           (lit(1993) + col("season_number") - 1).cast("string"),
                           lit("-"),
                           (lit(1994) + col("season_number") - 1).cast("string")
                       ))
    
    # Drop intermediate columns
    df = df.drop("days_diff", "season_start", "season_number")
    
    return df

# Apply the function to your dataframe
EREDIVISIE_HISTORY_df = add_season_columns(EREDIVISIE_HISTORY_df)

# Sort df by date field
EREDIVISIE_HISTORY_df = EREDIVISIE_HISTORY_df.sort(col("DATE").asc())

# Show the result
st.dataframe(EREDIVISIE_HISTORY_df)

### Feature Engineering

We'll be now calculating a set of features and consolidate match parameters in a unique table.
Features calculated will be about:
- Head to Head Matches
- Rolling Stats
- Average Metrics on Previous Matches

In [ ]:
def calculate_rolling_team_stats(df, num_matches=34):
    # Combine home and away games into a single dataframe
    home_games = df.select(
        col("SEASON"),
        col("DATE"),
        col("HOMETEAM").alias("TEAM"),
        col("FTHG").alias("GOALS_FOR"),
        col("FTAG").alias("GOALS_AGAINST"),
        col("FTR"),
        lit("H").alias("HOME_AWAY")
    )
    away_games = df.select(
        col("SEASON"),
        col("DATE"),
        col("AWAYTEAM").alias("TEAM"),
        col("FTAG").alias("GOALS_FOR"),
        col("FTHG").alias("GOALS_AGAINST"),
        col("FTR"),
        lit("A").alias("HOME_AWAY")
    )
    
    all_games = home_games.union_all(away_games)
    
    # Create a window function to order matches by date for each team
    window = Window.partition_by("TEAM").order_by("DATE")
    
    # Calculate match results
    all_games = all_games.with_columns([
        "WIN", "LOSS", "DRAW"
    ], [
        when((col("HOME_AWAY") == "H") & (col("FTR") == "H"), 1)
        .when((col("HOME_AWAY") == "A") & (col("FTR") == "A"), 1)
        .otherwise(0),
        
        when((col("HOME_AWAY") == "H") & (col("FTR") == "A"), 1)
        .when((col("HOME_AWAY") == "A") & (col("FTR") == "H"), 1)
        .otherwise(0),
        
        when(col("FTR") == "D", 1).otherwise(0)
    ])
    
    # Calculate rolling statistics
    rolling_stats = all_games.with_columns([
        f"WINS_LAST{num_matches}", 
        f"LOSSES_LAST{num_matches}", 
        f"DRAWS_LAST{num_matches}", 
        f"GOALS_FOR_LAST{num_matches}", 
        f"GOALS_AGAINST_LAST{num_matches}"
    ], [
        sum_("WIN").over(window.rows_between(-(num_matches-1), 0)),
        sum_("LOSS").over(window.rows_between(-(num_matches-1), 0)),
        sum_("DRAW").over(window.rows_between(-(num_matches-1), 0)),
        sum_("GOALS_FOR").over(window.rows_between(-(num_matches-1), 0)),
        sum_("GOALS_AGAINST").over(window.rows_between(-(num_matches-1), 0))
    ])
    
    # Calculate win percentage
    rolling_stats = rolling_stats.with_column(
        f"WIN_PERCENTAGE_LAST{num_matches}",
        col(f"WINS_LAST{num_matches}") / (col(f"WINS_LAST{num_matches}") + col(f"LOSSES_LAST{num_matches}") + col(f"DRAWS_LAST{num_matches}"))
    )
    
    return rolling_stats

# Calculate rolling stats for last 34 matches
rolling_stats_34 = calculate_rolling_team_stats(EREDIVISIE_HISTORY_df, 34)
# Show the results
st.text("\n--- Rolling Team Statistics for Last 34 Matches ---")
st.dataframe(rolling_stats_34.sort(["DATE", "TEAM"], ascending=[False, True]).limit(50))

# If you want to calculate for last 68 matches as well
st.text("\n--- Rolling Team Statistics for Last 68 Matches ---")
rolling_stats_68 = calculate_rolling_team_stats(EREDIVISIE_HISTORY_df, 68)
st.dataframe(rolling_stats_68.sort(["DATE", "TEAM"], ascending=[False, True]).limit(50))


In [ ]:
# Combine the results into 1 table
combined_rolling_stats = rolling_stats_34.join(
    rolling_stats_68,
    (rolling_stats_34["TEAM"] == rolling_stats_68["TEAM"]) & (rolling_stats_34["DATE"] == rolling_stats_68["DATE"]), 
    "left").select(
    rolling_stats_34["DATE"].alias("DATE"),
    rolling_stats_34["SEASON"].alias("SEASON"),
    rolling_stats_34["TEAM"].alias("HOMETEAM"),
    rolling_stats_34["WINS_LAST34"].alias("HOME_WINS_LAST34"),
    rolling_stats_34["LOSSES_LAST34"].alias("HOME_LOSSES_LAST34"),
    rolling_stats_34["DRAWS_LAST34"].alias("HOME_DRAWS_LAST34"),
    rolling_stats_34["GOALS_FOR_LAST34"].alias("HOME_GOALS_FOR_LAST34"),
    rolling_stats_34["GOALS_AGAINST_LAST34"].alias("HOME_GOALS_AGAINST_LAST34"),
    rolling_stats_34["WIN_PERCENTAGE_LAST34"].alias("HOME_WIN_PERCENTAGE_LAST34"),
    rolling_stats_68["WINS_LAST68"].alias("HOME_WINS_LAST68"),
    rolling_stats_68["LOSSES_LAST68"].alias("HOME_LOSSES_LAST68"),
    rolling_stats_68["DRAWS_LAST68"].alias("HOME_DRAWS_LAST68"),
    rolling_stats_68["GOALS_FOR_LAST68"].alias("HOME_GOALS_FOR_LAST68"),
    rolling_stats_68["GOALS_AGAINST_LAST68"].alias("HOME_GOALS_AGAINST_LAST68"),
    rolling_stats_68["WIN_PERCENTAGE_LAST68"].alias("HOME_WIN_PERCENTAGE_LAST68")
).sort(["DATE", "HOMETEAM"])

# Show the results
st.dataframe(combined_rolling_stats.order_by(col("DATE"), ascending=False).limit(100))

#result.write.mode("overwrite").save_as_table("TEAM_AVERAGES")

In [ ]:
def calc_h2h_opp_turf_specific_stats(df, num_matches=5):
    # Create a window function to order matches by date for each home team against each away team
    window = Window.partition_by("HOMETEAM", "AWAYTEAM").order_by("DATE")
    
    # Calculate match results
    h2h_stats = df.with_columns([
        "WIN", "LOSS", "DRAW", "GOALS_FOR", "GOALS_AGAINST"
    ], [
        when(col("FTR") == "H", 1).otherwise(0),
        when(col("FTR") == "A", 1).otherwise(0),
        when(col("FTR") == "D", 1).otherwise(0),
        col("FTHG"),
        col("FTAG")
    ])
    
    # Calculate rolling statistics
    rolling_stats = h2h_stats.with_columns([
        f"WINS_LAST{num_matches}", 
        f"LOSSES_LAST{num_matches}", 
        f"DRAWS_LAST{num_matches}", 
        f"GOALS_FOR_LAST{num_matches}", 
        f"GOALS_AGAINST_LAST{num_matches}"
    ], [
        sum_("WIN").over(window.rows_between(-(num_matches-1), 0)),
        sum_("LOSS").over(window.rows_between(-(num_matches-1), 0)),
        sum_("DRAW").over(window.rows_between(-(num_matches-1), 0)),
        sum_("GOALS_FOR").over(window.rows_between(-(num_matches-1), 0)),
        sum_("GOALS_AGAINST").over(window.rows_between(-(num_matches-1), 0))
    ])
    
    # Calculate win percentage
    rolling_stats = rolling_stats.with_column(
        f"WIN_PERCENTAGE_LAST{num_matches}",
        col(f"WINS_LAST{num_matches}") / (col(f"WINS_LAST{num_matches}") + col(f"LOSSES_LAST{num_matches}") + col(f"DRAWS_LAST{num_matches}"))
    )
    
    return rolling_stats

# Calculate head-to-head stats for last 5 matches
h2h_opp_turf_specific_stats_5 = calc_h2h_opp_turf_specific_stats(EREDIVISIE_HISTORY_df, 5)

# Calculate head-to-head stats for last 10 matches
h2h_opp_turf_specific_stats_10 = calc_h2h_opp_turf_specific_stats(EREDIVISIE_HISTORY_df, 10)

# Combine the results into 1 table
h2h_opp_turf_specific_stats = h2h_opp_turf_specific_stats_5.join(
    h2h_opp_turf_specific_stats_10,
    (h2h_opp_turf_specific_stats_5["HOMETEAM"] == h2h_opp_turf_specific_stats_10["HOMETEAM"]) & 
    (h2h_opp_turf_specific_stats_5["AWAYTEAM"] == h2h_opp_turf_specific_stats_10["AWAYTEAM"]) & 
    (h2h_opp_turf_specific_stats_5["DATE"] == h2h_opp_turf_specific_stats_10["DATE"]), 
    "left"
).select(
    h2h_opp_turf_specific_stats_5["DATE"].alias("DATE"),
    h2h_opp_turf_specific_stats_5["SEASON"].alias("SEASON"),
    h2h_opp_turf_specific_stats_5["HOMETEAM"].alias("HOMETEAM"),
    h2h_opp_turf_specific_stats_5["AWAYTEAM"].alias("AWAYTEAM"),
    h2h_opp_turf_specific_stats_5["WINS_LAST5"].alias("H2H_HOME_WINS_LAST5"),
    h2h_opp_turf_specific_stats_5["LOSSES_LAST5"].alias("H2H_HOME_LOSSES_LAST5"),
    h2h_opp_turf_specific_stats_5["DRAWS_LAST5"].alias("H2H_HOME_DRAWS_LAST5"),
    h2h_opp_turf_specific_stats_5["GOALS_FOR_LAST5"].alias("H2H_HOME_GOALS_FOR_LAST5"),
    h2h_opp_turf_specific_stats_5["GOALS_AGAINST_LAST5"].alias("H2H_HOME_GOALS_AGAINST_LAST5"),
    h2h_opp_turf_specific_stats_5["WIN_PERCENTAGE_LAST5"].alias("H2H_HOME_WIN_PERCENTAGE_LAST5"),
    h2h_opp_turf_specific_stats_10["WINS_LAST10"].alias("H2H_HOME_WINS_LAST10"),
    h2h_opp_turf_specific_stats_10["LOSSES_LAST10"].alias("H2H_HOME_LOSSES_LAST10"),
    h2h_opp_turf_specific_stats_10["DRAWS_LAST10"].alias("H2H_HOME_DRAWS_LAST10"),
    h2h_opp_turf_specific_stats_10["GOALS_FOR_LAST10"].alias("H2H_HOME_GOALS_FOR_LAST10"),
    h2h_opp_turf_specific_stats_10["GOALS_AGAINST_LAST10"].alias("H2H_HOME_GOALS_AGAINST_LAST10"),
    h2h_opp_turf_specific_stats_10["WIN_PERCENTAGE_LAST10"].alias("H2H_HOME_WIN_PERCENTAGE_LAST10")
).sort(["DATE", "HOMETEAM", "AWAYTEAM"])

# Show the results
st.text("\n--- Head-to-Head Rolling Statistics ---")
st.dataframe(h2h_opp_turf_specific_stats.limit(10))

# Count total number of rows
total_rows = h2h_opp_turf_specific_stats.count()
st.text(f"\nTotal number of matches: {total_rows}")

In [ ]:
# Filter for matches between Go Ahead Eagles and Vitesse
gae_vitesse_matches = EREDIVISIE_HISTORY_df.filter((col("HOMETEAM") == "Go Ahead Eagles") & (col("AWAYTEAM") == "Vitesse") |
                                (col("HOMETEAM") == "Vitesse") & (col("AWAYTEAM") == "Go Ahead Eagles"))

# Display the results
# gae_vitesse_matches.sort("DATE", "HOMETEAM").show(100)


# Filter for matches between Go Ahead Eagles and Vitesse
gae_vitesse_combined_h2h_stats = h2h_opp_turf_specific_stats.filter((col("HOMETEAM") == "Go Ahead Eagles") & (col("AWAYTEAM") == "Vitesse") |
                                (col("HOMETEAM") == "Vitesse") & (col("AWAYTEAM") == "Go Ahead Eagles"))

# Display the results
st.dataframe(gae_vitesse_combined_h2h_stats.sort("DATE", "HOMETEAM").limit(100))


# Calculate some statistics
total_matches = gae_vitesse_matches.count()
gae_home_matches = gae_vitesse_matches.filter(col("HOMETEAM") == "Go Ahead Eagles").count()
vitesse_home_matches = gae_vitesse_matches.filter(col("HOMETEAM") == "Vitesse").count()

In [ ]:
# Perform join #1, on the results of the last 30 and 60 matches 
EREDIVISIE_HISTORY_combined_1_df = EREDIVISIE_HISTORY_df.join(
    combined_rolling_stats,["DATE", "SEASON", "HOMETEAM"])

# Perform join #2, on the results of the last matches against the specific opponents
EREDIVISIE_HISTORY_combined_2_df = EREDIVISIE_HISTORY_combined_1_df.join(
    h2h_opp_turf_specific_stats, ["DATE", "SEASON", "HOMETEAM", "AWAYTEAM"], 
    rsuffix="_right")

# Create a GAME_OUTCOME field based on FTR for later use
EREDIVISIE_HISTORY_combined_2_df = EREDIVISIE_HISTORY_combined_2_df.with_columns(
    ["GAME_OUTCOME"],
    [when(col("FTR") == "D", 0)
     .when(col("FTR") == "H", 1)
     .when(col("FTR") == "A", 2)
     .otherwise(None)]  # This handles any unexpected values in FTR
)

print("\n--- EREDIVISIE_HISTORY_combined_2_df ---")
print("Count:", EREDIVISIE_HISTORY_combined_2_df.count())
st.dataframe(EREDIVISIE_HISTORY_combined_2_df.sort(col("date"), ascending=False).limit(50))

In [ ]:
EREDIVISIE_FEATURES_df = EREDIVISIE_HISTORY_combined_2_df.with_column('DATE', F.to_varchar(F.col('DATE'), 'YYYY-MM-DD'))
EREDIVISIE_FEATURES_df.write.save_as_table(f'eredivisie_features', mode='overwrite')

session.table(f'eredivisie_features').sort(F.col('date'), ascending = False).limit(50)

Our final dataset contains the following features:

| Feature                              | Description                                                                                             |
|--------------------------------------|---------------------------------------------------------------------------------------------------------|
| DATE                                 | The date of the match.                                                                                 |
| SEASON                               | The season in which the match is played.                                                               |
| HOMETEAM                             | The name of the home team.                                                                              |
| AWAYTEAM                             | The name of the away team.                                                                              |
| H2H_HOME_WINS_LAST5                 | Number of wins for the home team in the last 5 games against the opponent.                             |
| H2H_HOME_LOSSES_LAST5               | Number of losses for the home team in the last 5 games against the opponent.                           |
| H2H_HOME_DRAWS_LAST5                | Number of draws for the home team in the last 5 games against the opponent.                            |
| H2H_HOME_GOALS_FOR_LAST5            | Total goals scored by the home team in the last 5 games against the opponent.                          |
| H2H_HOME_GOALS_AGAINST_LAST5        | Total goals conceded by the home team in the last 5 games against the opponent.                        |
| H2H_HOME_WIN_PERCENTAGE_LAST5       | Win percentage of the home team in the last 5 games against the opponent.                              |
| H2H_HOME_WINS_LAST10                | Number of wins for the home team in the last 10 games against the opponent.                            |
| H2H_HOME_LOSSES_LAST10              | Number of losses for the home team in the last 10 games against the opponent.                          |
| H2H_HOME_DRAWS_LAST10               | Number of draws for the home team in the last 10 games against the opponent.                           |
| H2H_HOME_GOALS_FOR_LAST10           | Total goals scored by the home team in the last 10 games against the opponent.                         |
| H2H_HOME_GOALS_AGAINST_LAST10       | Total goals conceded by the home team in the last 10 games against the opponent.                       |
| H2H_HOME_WIN_PERCENTAGE_LAST10      | Win percentage of the home team in the last 10 games against the opponent.                             |
| HOME_WINS_LAST34                    | Number of wins for the home team in the last 34 games.                                                |
| HOME_LOSSES_LAST34                  | Number of losses for the home team in the last 34 games.                                              |
| HOME_DRAWS_LAST34                   | Number of draws for the home team in the last 34 games.                                               |
| HOME_GOALS_FOR_LAST34               | Total goals scored by the home team in the last 34 games.                                             |
| HOME_GOALS_AGAINST_LAST34           | Total goals conceded by the home team in the last 34 games.                                           |
| HOME_WIN_PERCENTAGE_LAST34          | Win percentage of the home team in the last 34 games.                                                 |
| HOME_WINS_LAST68                    | Number of wins for the home team in the last 68 games.                                                |
| HOME_LOSSES_LAST68                  | Number of losses for the home team in the last 68 games.                                              |
| HOME_DRAWS_LAST68                   | Number of draws for the home team in the last 68 games.                                               |
| HOME_GOALS_FOR_LAST68               | Total goals scored by the home team in the last 68 games.                                             |
| HOME_GOALS_AGAINST_LAST68           | Total goals conceded by the home team in the last 68 games.                                           |
| HOME_WIN_PERCENTAGE_LAST68          | Win percentage of the home team in the last 68 games.                                                 |

